In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns

In [2]:
from simpledbf import Dbf5


In [3]:
#conapo
conapo = pd.read_csv("datos-crudos/pob_mit_proyecciones.csv",encoding="latin1")
conapo = conapo[conapo.EDAD < 18 ]
conapo.CVE_GEO = conapo.CVE_GEO.apply(str).apply(lambda x :x.zfill(2))
conapo.drop(["RENGLON"],axis=1, inplace=True)

In [4]:
data_conapo = conapo[(conapo.AÑO <= 2021) & (conapo.AÑO >= 2012)].reset_index(drop=True)
data_conapo.SEXO = data_conapo.SEXO.map({"Hombres": 1, "Mujeres":2})

In [5]:
data_conapo.drop("ENTIDAD",axis=1).to_csv("datos-limpios/poblaciones_conapo.csv",index=False)

In [6]:
conapo[["ENTIDAD","CVE_GEO"]].drop_duplicates().set_index("CVE_GEO").ENTIDAD.to_dict()

{'00': 'República Mexicana',
 '01': 'Aguascalientes',
 '02': 'Baja California',
 '03': 'Baja California Sur',
 '04': 'Campeche',
 '05': 'Coahuila',
 '06': 'Colima',
 '07': 'Chiapas',
 '08': 'Chihuahua',
 '09': 'Ciudad de México',
 '10': 'Durango',
 '11': 'Guanajuato',
 '12': 'Guerrero',
 '13': 'Hidalgo',
 '14': 'Jalisco',
 '15': 'México',
 '16': 'Michoacán',
 '17': 'Morelos',
 '18': 'Nayarit',
 '19': 'Nuevo León',
 '20': 'Oaxaca',
 '21': 'Puebla',
 '22': 'Querétaro',
 '23': 'Quintana Roo',
 '24': 'San Luis Potosí',
 '25': 'Sinaloa',
 '26': 'Sonora',
 '27': 'Tabasco',
 '28': 'Tamaulipas',
 '29': 'Tlaxcala',
 '30': 'Veracruz',
 '31': 'Yucatán',
 '32': 'Zacatecas'}

De toda la base hicimos un primer filtrado por
la variable EDAD_AGRU para elegir los brupos de edad menores a 20 años. Posteriormente
se hizo un segundo filtro para seleccionar a las personas que contaran con precisión en un su
fecha de nacimiento y en la fecha de ocurrencia. Se analizó el conjunto de datos con estas especificaciones

conapo, poblaciones a mitad de año 
https://datos.gob.mx/busca/dataset/proyecciones-de-la-poblacion-de-mexico-y-de-las-entidades-federativas-2016-2050/resource/353817d4-ac9d-4709-bbd9-52936fe0423f?inner_span=True

mortalidad inegi 

https://www.inegi.org.mx/programas/mortalidad/?ps=Microdatos


# base 2021

In [7]:
capgpo = Dbf5("./datos-crudos/defunciones_base_datos_2021_dbf/CAPGPO.dbf",codec="latin1")
capgpo = capgpo.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_esp = capgpo[capgpo.GPO.isna()!=True].reset_index(drop=True)
capgpo_esp["CVE_COMPUESTA"] = capgpo_esp.CAP + capgpo_esp.GPO
dic_capgpo = capgpo_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [8]:
dbf = Dbf5("./datos-crudos/defunciones_base_datos_2021_dbf/defun21.dbf")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2021.csv")

# Base 2020

In [9]:
capgpo_2020 = Dbf5("./datos-crudos/defunciones_base_datos_2020_dbf/CAPGPO.dbf",codec="latin1")
capgpo_2020 = capgpo_2020.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2020_esp = capgpo_2020[capgpo_2020.GPO.isna()!=True].reset_index(drop=True)
capgpo_2020_esp["CVE_COMPUESTA"] = capgpo_2020_esp.CAP + capgpo_2020_esp.GPO
dic_capgpo = capgpo_2020_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [10]:
ef = capgpo_2020_esp == capgpo_esp

In [11]:
dbf = Dbf5("./datos-crudos/defunciones_base_datos_2020_dbf/defun20.dbf")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2020.csv")

In [12]:
data

,ENT_OCURR,MUN_OCURR,SEXO,CAPITULO,GRUPO,DIA_NACIM,MES_NACIM,ANIO_NACIM,DIA_OCURR,MES_OCURR,ANIO_OCUR,EDAD_AGRU,FECHA_NACIMIENTO,FECHA_OCURRENCIA,DELTA_DIAS,EDAD_ANIOS,EDAD,CVEGEO,CVE_CAPGPO,CAPGPO
0,01,001,2,10,2,15,7,2019,16,1,2020,01,2019-07-15,2020-01-16,185 days,0.506513,0,01001,1002,Influenza (gripe) y neumon¡a (J09-J18)
1,01,001,1,17,11,21,1,2020,2,2,2020,01,2020-01-21,2020-02-02,12 days,0.032855,0,01001,1711,"Anomal¡as cromos¢micas, no clasificadas en otr..."
2,01,011,1,10,3,8,12,2019,24,1,2020,01,2019-12-08,2020-01-24,47 days,0.128682,0,01011,1003,Otras infecciones agudas de las v¡as respirato...
3,01,002,2,6,11,19,4,2012,22,1,2020,06,2012-04-19,2020-01-22,2834 days,7.759228,7,01002,0611,Otros trastornos del sistema nervioso (G90-G99)
4,01,001,1,16,5,19,1,2020,23,1,2020,01,2020-01-19,2020-01-23,4 days,0.010952,0,01001,1605,Infecciones espec¡ficas del per¡odo perinatal ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33093,32,056,2,16,6,25,8,2020,27,8,2020,01,2020-08-25,2020-08-27,2 days,0.005476,0,32056,1606,Trastornos hemorr gicos y hematol¢gicos del fe...
33094,32,056,1,17,3,14,8,2020,26,12,2020,01,2020-08-14,2020-12-26,134 days,0.366880,0,32056,1703,Malformaciones congnitas del sistema circulat...
33095,32,056,1,2,17,28,12,2019,25,12,2020,01,2019-12-28,2020-12-25,363 days,0.993860,0,32056,0217,Tumores (neoplasias) benignos (D10-D36)
33096,32,010,2,1,4,4,7,2020,24,12,2020,01,2020-07-04,2020-12-24,173 days,0.473658,0,32010,0104,Otras enfermedades bacterianas (A30-A49)


# Base 2019

In [12]:
capgpo_2019 = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2019/CAPGPO.dbf",codec="latin1")
capgpo_2019 = capgpo_2019.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2019_esp = capgpo_2019[capgpo_2019.GPO.isna()!=True].reset_index(drop=True)
capgpo_2019_esp["CVE_COMPUESTA"] = capgpo_2019_esp.CAP + capgpo_2019_esp.GPO
dic_capgpo = capgpo_2019_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [13]:
dbf = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2019/DEFUN19.DBF")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2019.csv")

# Base 2018

In [14]:
capgpo_2018 = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2018/CAPGPO.dbf",codec="latin1")
capgpo_2018 = capgpo_2018.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2018_esp = capgpo_2018[capgpo_2018.GPO.isna()!=True].reset_index(drop=True)
capgpo_2018_esp["CVE_COMPUESTA"] = capgpo_2018_esp.CAP + capgpo_2018_esp.GPO
dic_capgpo = capgpo_2018_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [15]:
ef = capgpo_2018_esp == capgpo_esp
ef[ef.apply(lambda x : False in x ,axis=1)]

,CONSE,CAP,GPO,DESCRIP,CVE_COMPUESTA


In [16]:
dbf = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2018/DEFUN18.DBF")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2018.csv")

# Base 2017

In [17]:
capgpo_2017 = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2017/CAPGPO.dbf",codec="latin1")
capgpo_2017 = capgpo_2017.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2017_esp = capgpo_2017[capgpo_2017.GPO.isna()!=True].reset_index(drop=True)
capgpo_2017_esp["CVE_COMPUESTA"] = capgpo_2017_esp.CAP + capgpo_2017_esp.GPO
dic_capgpo = capgpo_2017_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [18]:
ef = capgpo_2017_esp == capgpo_esp
ef[ef.apply(lambda x : False in x ,axis=1)]

,CONSE,CAP,GPO,DESCRIP,CVE_COMPUESTA


In [19]:
dbf = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2017/DEFUN17.dbf")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2017.csv")

In [22]:
data.shape

(40616, 20)

# Base 2016

In [20]:
capgpo_2016 = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2016/CAPGPO.dbf",codec="latin1")
capgpo_2016 = capgpo_2016.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2016_esp = capgpo_2016[capgpo_2016.GPO.isna()!=True].reset_index(drop=True)
capgpo_2016_esp["CVE_COMPUESTA"] = capgpo_2016_esp.CAP + capgpo_2016_esp.GPO
dic_capgpo = capgpo_2016_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [24]:
ef = capgpo_2016_esp == capgpo_esp
ef[ef.apply(lambda x : False in x ,axis=1)]

,CONSE,CAP,GPO,DESCRIP,CVE_COMPUESTA


In [21]:
dbf = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2016/DEFUN16.dbf")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2016.csv")

# base 2015

In [22]:
capgpo_2015 = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2015/CAPGPO.dbf",codec="latin1")
capgpo_2015 = capgpo_2015.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2015_esp = capgpo_2015[capgpo_2015.GPO.isna()!=True].reset_index(drop=True)
capgpo_2015_esp["CVE_COMPUESTA"] = capgpo_2015_esp.CAP + capgpo_2015_esp.GPO
dic_capgpo = capgpo_2015_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [27]:
#hay una nueva cat 2201

In [23]:
dbf = Dbf5("./datos-crudos/defunciones_generales_base_datos_2015_2019_dbf/defunciones_base_datos_2015/DEFUN15.DBF")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2015.csv")

# Base 2014

In [24]:
capgpo_2014 = Dbf5("./datos-crudos/defunciones_generales_base_datos_2010_2014_dbf/defunciones_base_datos_2014_dbf/CAPGPO.dbf",codec="latin1")
capgpo_2014 = capgpo_2014.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2014_esp = capgpo_2014[capgpo_2014.GRUPO.isna()!=True].reset_index(drop=True)
capgpo_2014_esp["CVE_COMPUESTA"] = capgpo_2014_esp.CAPITULO.apply(str).apply(lambda x : x.zfill(2)) + capgpo_2014_esp.GRUPO.apply(str).apply(lambda x : x.zfill(2))
dic_capgpo = capgpo_2014_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [25]:
dbf = Dbf5("./datos-crudos/defunciones_generales_base_datos_2010_2014_dbf/defunciones_base_datos_2014_dbf/DEFUN14.dbf")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2014.csv")

In [31]:
df_fecha_incompleta.shape
df_fecha_completa.shape

(37070, 16)

In [32]:
eff

,ENT_OCURR,MUN_OCURR,SEXO,CAPITULO,GRUPO,DIA_NACIM,MES_NACIM,ANIO_NACIM,DIA_OCURR,MES_OCURR,ANIO_OCUR,EDAD_AGRU,EDAD,CVEGEO,CVE_CAPGPO,FECHA_OCURRENCIA,CAPGPO
0,01,007,2,20,17,99,6,2013,1,5,2014,01,0.0,01007,2017,01-5-2014,Otros accidentes que obstruyen la respiraci¢n ...
1,01,001,2,20,17,99,3,2014,3,5,2014,01,0.0,01001,2017,01-5-2014,Otros accidentes que obstruyen la respiraci¢n ...
2,01,001,2,17,6,99,5,2014,5,7,2014,01,0.0,01001,1706,01-7-2014,Otras malformaciones congnitas del sistema di...
3,01,001,1,1,4,99,6,2014,19,7,2014,01,0.0,01001,0104,01-7-2014,Otras enfermedades bacterianas (A30-A49)
4,01,001,1,1,4,99,8,2013,6,7,2014,01,0.0,01001,0104,01-7-2014,Otras enfermedades bacterianas (A30-A49)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5256,32,017,2,10,1,99,1,2014,2,2,2014,01,0.0,32017,1001,01-2-2014,Infecciones agudas de las v¡as respiratorias s...
5257,32,056,1,6,10,99,1,2014,10,12,2014,01,0.0,32056,0610,01-12-2014,Par lisis cerebral y otros s¡ndromes paral¡tic...
5258,32,017,2,3,6,99,1,2014,19,2,2014,01,0.0,32017,0306,01-2-2014,Ciertos trastornos que afectan el mecanismo de...
5259,32,028,1,10,2,99,11,2013,3,2,2014,01,0.0,32028,1002,01-2-2014,Influenza (gripe) y neumon¡a (J09-J18)


# Base 2013

In [26]:
capgpo_2013 = Dbf5("./datos-crudos/defunciones_generales_base_datos_2010_2014_dbf/defunciones_base_datos_2013_dbf/CAPGPO.dbf",codec="latin1")
capgpo_2013 = capgpo_2013.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2013_esp = capgpo_2013[capgpo_2013.GRUPO.isna()!=True].reset_index(drop=True)
capgpo_2013_esp["CVE_COMPUESTA"] = capgpo_2013_esp.CAPITULO.apply(str).apply(lambda x : x.zfill(2)) + capgpo_2013_esp.GRUPO.apply(str).apply(lambda x : x.zfill(2))
dic_capgpo = capgpo_2013_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [27]:
dbf = Dbf5("./datos-crudos/defunciones_generales_base_datos_2010_2014_dbf/defunciones_base_datos_2013_dbf/DEFUN13.DBF")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)  
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1) 
 
df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2013.csv")

# Base 2012

In [28]:
capgpo_2012 = Dbf5("./datos-crudos/defunciones_generales_base_datos_2010_2014_dbf/defunciones_base_datos_2012_dbf/CAPGPO.dbf",codec="latin1")
capgpo_2012 = capgpo_2012.to_dataframe()
#dic_capgpo = capgpo.set_index("CVE").DESCRIP.to_dict()
capgpo_2012_esp = capgpo_2012[capgpo_2012.GRUPO.isna()!=True].reset_index(drop=True)
capgpo_2012_esp["CVE_COMPUESTA"] = capgpo_2012_esp.CAPITULO.apply(str).apply(lambda x : x.zfill(2)) + capgpo_2012_esp.GRUPO.apply(str).apply(lambda x : x.zfill(2))
dic_capgpo = capgpo_2012_esp.set_index("CVE_COMPUESTA").DESCRIP.to_dict()

In [29]:
dbf = Dbf5("./datos-crudos/defunciones_generales_base_datos_2010_2014_dbf/defunciones_base_datos_2012_dbf/DEFUN12.dbf")
df = dbf.to_dataframe()

# ENTIDAD; MUNICIPIO; SEXO; CAUSA; EDAD
df = df[["ENT_OCURR","MUN_OCURR","SEXO", 'CAPITULO','GRUPO',"DIA_NACIM","MES_NACIM","ANIO_NACIM","DIA_OCURR","MES_OCURR","ANIO_OCUR","EDAD_AGRU"]]
df = df[df.EDAD_AGRU.apply(lambda x : int(x)<=8)].reset_index(drop=True)
df_fecha_incompleta = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1)]
df_fecha_completa = df[df.apply(lambda x : 99 in x[["DIA_NACIM","MES_NACIM","DIA_OCURR","MES_OCURR"]].values or 9999 in x[["ANIO_NACIM","ANIO_OCUR"]].values, axis=1).apply(lambda x : x==False)].reset_index().drop("index",axis=1)



#tratamiento de fe chas
fechas_incorrectas = {
    '29-2-1997':'28-2-1997', 
    '29-2-1993':'28-2-1993' , 
    '29-2-1994':'28-2-1994', 
    '30-2-1997':'28-2-1997',
    '31-4-2012': '30-4-2012'
}

df_fecha_completa["FECHA_NACIMIENTO"] = df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1).apply(lambda x: x if x not in fechas_incorrectas.keys() else fechas_incorrectas[x])
df_fecha_completa["FECHA_OCURRENCIA"] = df_fecha_completa.apply(lambda x : str(x.DIA_OCURR)+ "-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1).apply(lambda x: x if x not in fechas_incorrectas.keys() else fechas_incorrectas[x])

df_fecha_completa["FECHA_NACIMIENTO"] = pd.to_datetime(df_fecha_completa.FECHA_NACIMIENTO, format="%d-%m-%Y")
df_fecha_completa["FECHA_OCURRENCIA"] = pd.to_datetime(df_fecha_completa.FECHA_OCURRENCIA, format="%d-%m-%Y")

df_fecha_completa["DELTA_DIAS"] = df_fecha_completa.FECHA_OCURRENCIA -  df_fecha_completa.FECHA_NACIMIENTO
df_fecha_completa["EDAD_ANIOS"] = df_fecha_completa.DELTA_DIAS/np.timedelta64(1, 'Y')

#data que nos interesa 
data = df_fecha_completa[df_fecha_completa.EDAD_ANIOS<18].reset_index(drop=True)
data["EDAD"] = data.EDAD_ANIOS.apply(int)
data["CVEGEO"] = data.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
data["CVE_CAPGPO"] = data.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
data["CAPGPO"] = data["CVE_CAPGPO"].map(dic_capgpo)

# Exportamiento
eff = df_fecha_incompleta[(df_fecha_incompleta.MES_OCURR.apply(int)!=99) &(df_fecha_incompleta.ANIO_OCUR.apply(int)!=9999)]
eff = eff[eff.EDAD_AGRU.apply(int)<=6].reset_index(drop=True)
eff["EDAD"] = eff.EDAD_AGRU.apply(int).map({1: 0, 2: 1, 3: 2, 4:3, 5: 4})
eff["CVEGEO"] = eff.apply(lambda x : x.ENT_OCURR + x.MUN_OCURR, axis=1)
eff["CVE_CAPGPO"] = eff.apply(lambda x : str(x.CAPITULO).zfill(2) + str(x.GRUPO).zfill(2),axis=1)
eff["FECHA_OCURRENCIA"] = pd.to_datetime(eff.apply(lambda x : "01-" +str(x.MES_OCURR)+ "-" +str(x.ANIO_OCUR),axis=1), format="%d-%m-%Y")
eff["CAPGPO"] = eff["CVE_CAPGPO"].map(dic_capgpo)
pd.concat([data[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']],eff[['ENT_OCURR', 'MUN_OCURR', 'SEXO',
       'EDAD_AGRU', 'FECHA_OCURRENCIA', 'EDAD', 'CVEGEO', 'CVE_CAPGPO', 'CAPGPO']]]).reset_index(drop=True).to_csv("datos-limpios/defs_2012.csv")

In [42]:
ef

,ENT_OCURR,MUN_OCURR,SEXO,EDAD_AGRU,FECHA_OCURRENCIA,EDAD,CVEGEO,CVE_CAPGPO,CAPGPO
0,01,001,2,07,2011-12-16 00:00:00,13.0,01001,1701,Malformaciones congnitas del sistema nervioso...
1,01,001,2,01,2011-09-21 00:00:00,0.0,01001,1605,Infecciones espec¡ficas del per¡odo perinatal ...
2,01,001,2,01,2011-12-24 00:00:00,0.0,01001,1604,Trastornos respiratorios y cardiovasculares es...
3,01,001,2,01,2011-12-28 00:00:00,0.0,01001,1703,Malformaciones congnitas del sistema circulat...
4,01,001,1,02,2011-12-21 00:00:00,1.0,01001,0111,Infecciones virales caracterizadas por lesione...
...,...,...,...,...,...,...,...,...,...
5966,32,051,1,01,01-12-2012,0.0,32051,2005,Ocupante de autom¢vil lesionado en accidente d...
5967,32,014,1,01,01-10-2012,0.0,32014,2016,Ahogamiento y sumersi¢n accidentales (W65-W74)
5968,32,056,2,01,01-5-2012,0.0,32056,0104,Otras enfermedades bacterianas (A30-A49)
5969,32,056,1,01,01-8-2012,0.0,32056,0101,Enfermedades infecciosas intestinales (A00-A09)


In [ ]:
eff.shape

In [ ]:
ef = df_fecha_completa.FECHA_OCURRENCIA.apply(lambda x : pd.to_datetime(x, format="%d-%m-%Y", errors='coerce'))

In [ ]:
df_fecha_completa[ef.isna()].FECHA_OCURRENCIA.values

In [ ]:
fechas_incorrectas.values()

In [ ]:
fechas_incorrectas = {
    '29-2-1997':'28-2-1997', 
    '29-2-1993':'28-2-1993' , 
    '29-2-1994':'28-2-1994', 
    '30-2-1997':'28-2-1997',
    '31-4-2012': '30-4-2012'
}
df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1).apply(lambda x: x if x not in fechas_incorrectas.keys() else fechas_incorrectas[x])

In [ ]:
fechas_incorrectas

In [ ]:
df_fecha_completa.apply(lambda x : str(x.DIA_NACIM)+ "-" +str(x.MES_NACIM)+ "-" +str(x.ANIO_NACIM),axis=1)